In [3]:
from google.cloud import bigquery
from google.cloud.exceptions import BadRequest, NotFound, GoogleCloudError
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hackathon-agents-044c975e8972.json"
client = bigquery.Client(project="hackathon-agents")

##### 1. Creating a JSON metadata of the tables present in the BQ database

In [5]:
from google.cloud import bigquery
import os
import json

# Set the environment variable for Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hackathon-agents-044c975e8972.json"

# Initialize the BigQuery client
client = bigquery.Client(project="hackathon-agents")

# Define the dataset
dataset_id = "StyleHub"  # Only the dataset ID, not the full path

# Example primary keys metadata
# This is an example structure for primary keys. Adjust this to fit your actual metadata.
primary_keys = {
    "table1": ["column1", "column2"],
    "table2": ["column3"],
    # Add other tables and their primary keys here
}

print(f"Using project: {client.project}")
print(f"Using dataset: {dataset_id}")

try:
    # Get the dataset reference
    dataset_ref = client.dataset(dataset_id)
    dataset = client.get_dataset(dataset_ref)

    # Initialize the JSON structure
    dataset_info = {"dataset_id": dataset_id, "tables": []}

    # List all tables in the dataset
    tables = list(client.list_tables(dataset))

    for table in tables:
        # Get the table reference
        table_ref = dataset_ref.table(table.table_id)
        table = client.get_table(table_ref)

        # Initialize the table info
        table_info = {
            "table_name": table.table_id,
            "table_description": table.description or "",
            "columns": [],
        }

        # Iterate over the schema to get column details
        for schema_field in table.schema:
            column_info = {
                "column_name": schema_field.name,
                "column_type": schema_field.field_type,
                "column_description": schema_field.description or "",
                "is_primary_key": schema_field.name
                in primary_keys.get(table.table_id, []),
            }
            table_info["columns"].append(column_info)

        # Add the table info to the dataset info
        dataset_info["tables"].append(table_info)

    # Export the dataset info to a JSON file
    with open("dataset_info1.json", "w") as json_file:
        json.dump(dataset_info, json_file, indent=4)

    print("Dataset information has been exported to dataset_info.json")

except NotFound:
    print("Dataset not found. Please check the dataset ID and project ID.")
except Exception as e:
    print(f"An error occurred: {e}")

Using project: hackathon-agents
Using dataset: StyleHub
Dataset information has been exported to dataset_info.json


##### 2. Listing out the count of unique values and total values in the DB

In [ ]:
from google.cloud import bigquery
import os

# Set the environment variable for Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hackathon-agents-044c975e8972.json"

# Initialize the BigQuery client
client = bigquery.Client(project="hackathon-agents")


def get_table_statistics(dataset_id):
    dataset_ref = client.dataset(dataset_id)
    tables = list(client.list_tables(dataset_ref))

    dataset_stats = {}

    for table in tables:

        table_id = table.table_id
        print("table:> ", table_id)
        table_ref = dataset_ref.table(table_id)
        table = client.get_table(table_ref)

        table_stats = {}

        for schema_field in table.schema:
            column_name = schema_field.name
            column_type = schema_field.field_type

            if column_type == "GEOGRAPHY":
                print(f"Skipping column {column_name} of type GEOGRAPHY")
                continue

            # SQL to get the total count and distinct count of entries
            query = f"""
            SELECT 
                COUNT(*) AS total_count,
                COUNT(DISTINCT {column_name}) AS unique_count
            FROM `{client.project}.{dataset_id}.{table_id}`
            """

            query_job = client.query(query)
            result = query_job.result().to_dataframe().iloc[0]

            total_count = result["total_count"]
            unique_count = result["unique_count"]

            table_stats[column_name] = {
                "total_entries": total_count,
                "unique_values_count": unique_count,
            }

        dataset_stats[table_id] = table_stats

    return dataset_stats


# Example usage
dataset_id = "StyleHub"  # Your dataset ID

dataset_statistics = get_table_statistics(dataset_id)

# Print the statistics
for table, stats in dataset_statistics.items():
    print(f"Table: {table}")
    for column, col_stats in stats.items():
        print(f"  Column: {column}")
        print(f"    Total Entries: {col_stats['total_entries']}")
        print(f"    Unique Values Count: {col_stats['unique_values_count']}")

table:>  distribution_centers


c:\Users\ashis\miniconda3\envs\hack1\lib\site-packages\google\cloud\bigquery\table.py:1962: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Skipping column distribution_center_geom of type GEOGRAPHY
table:>  events
table:>  inventory_items
table:>  order_items
table:>  orders
table:>  products
table:>  users
Skipping column user_geom of type GEOGRAPHY
Table: distribution_centers
  Column: id
    Total Entries: 10
    Unique Values Count: 10
  Column: name
    Total Entries: 10
    Unique Values Count: 10
  Column: latitude
    Total Entries: 10
    Unique Values Count: 10
  Column: longitude
    Total Entries: 10
    Unique Values Count: 10
Table: events
  Column: id
    Total Entries: 2426128
    Unique Values Count: 2426128
  Column: user_id
    Total Entries: 2426128
    Unique Values Count: 80018
  Column: sequence_number
    Total Entries: 2426128
    Unique Values Count: 13
  Column: session_id
    Total Entries: 2426128
    Unique Values Count: 681434
  Column: created_at
    Total Entries: 2426128
    Unique Values Count: 2185660
  Column: ip_address
    Total Entries: 2426128
    Unique Values Count: 681372
  Colu

##### 3. Vectorizing the JSON data into Chroma DB

In [19]:
# CONVERT THE JSON FILE TO PARAGRAPHS DATASET
import json
def json_to_paragraphs(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    paragraphs = []
    
    for table in data.get('tables', []):
        table_name = table.get('table_name', 'Unnamed Table')
        table_description = table.get('table_description', 'No description available.')
        
        paragraph = f"Table '{table_name}': {table_description}\n"
        paragraph += "Columns:\n"
        
        for column in table.get('columns', []):
            column_name = column.get('column_name', 'Unnamed Column')
            column_type = column.get('column_type', 'Unknown Type')
            column_description = column.get('column_description', 'No description available.')
            is_primary_key = column.get('is_primary_key', False)
            primary_key_info = " (Primary Key)" if is_primary_key else ""
            
            foreign_key_info = ""
            if 'foreign_key' in column:
                fk_table = column['foreign_key'].get('reference_table', 'Unknown Table')
                fk_column = column['foreign_key'].get('reference_column', 'Unknown Column')
                foreign_key_info = f" (Foreign Key references {fk_table}.{fk_column})"
                
            paragraph += f"  - {column_name} ({column_type}): {column_description}{primary_key_info}{foreign_key_info}\n"
        
        paragraphs.append(paragraph)
    
    return paragraphs

# Example usage
file_path = 'dataset_info.json'
paragraphs = json_to_paragraphs(file_path)
for paragraph in paragraphs:
    print(paragraph)
    print('>>>'*20)


Table 'distribution_centers': Stores information about the distribution centers for the StyleHub website, including their geographical locations.
Columns:
  - id (INTEGER): Unique identifier for each distribution center. (Primary Key)
  - name (STRING): Name of the distribution center.
  - latitude (FLOAT): Geographical latitude of the distribution center.
  - longitude (FLOAT): Geographical longitude of the distribution center.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Table 'events': Logs web events generated by users on the StyleHub website.
Columns:
  - id (INTEGER): Unique identifier for each event. (Primary Key)
  - user_id (INTEGER): Identifier for the user who generated the event. References the 'id' column in the 'users' table. (Foreign Key references users.id)
  - sequence_number (INTEGER): Sequential number for the event in the user's session.
  - session_id (STRING): Identifier for the user session during which the event was generated.
  - created_at (TI

In [20]:
import chromadb
import numpy as np
import pandas as pd
from chromadb import Documents, EmbeddingFunction, Embeddings

In [21]:
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [22]:
for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [23]:
from google.genai import types


class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        EMBEDDING_MODEL_ID = "models/gemini-embedding-exp-03-07"  # @param ["models/embedding-001", "models/text-embedding-004", "models/gemini-embedding-exp-03-07", "models/gemini-embedding-exp"] {"allow-input": true, "isTemplate": true}
        title = "Database Schema Embedding"
        response = client.models.embed_content(
            model=EMBEDDING_MODEL_ID,
            contents=input,
            config=types.EmbedContentConfig(
                task_type="retrieval_document", title=title
            ),
        )

        return response.embeddings[0].values

In [24]:
def create_chroma_db(documents, name):
    chroma_client = chromadb.Client()
    db = chroma_client.create_collection(
        name=name, embedding_function=GeminiEmbeddingFunction()
    )

    for i, d in enumerate(documents):
        db.add(documents=d, ids=str(i))
    return db

In [25]:
db = create_chroma_db(paragraphs, "StyleHubMetadataDB")

C:\Users\ashis\AppData\Local\Temp\ipykernel_5088\2697842865.py:4: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  name=name, embedding_function=GeminiEmbeddingFunction()


In [26]:
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:3],
    "Documents": sample_data['documents'][:3],
    "Embeddings": [str(emb)[:50] + "..." for emb in sample_data['embeddings'][:3]]  # Truncate embeddings
})

print(df)

  IDs                                          Documents  \
0   0  Table 'distribution_centers': Stores informati...   
1   1  Table 'events': Logs web events generated by u...   
2   2  Table 'inventory_items': Tracks inventory item...   

                                          Embeddings  
0  [-0.02675541 -0.00857388  0.0068732  ... -0.01...  
1  [-0.03233003  0.01725394 -0.00741226 ... -0.00...  
2  [-0.02490733  0.00380711  0.01398553 ... -0.01...  


In [27]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [28]:
passage = get_relevant_passage("touch screen features", db)

In [30]:
print(passage)

Table 'distribution_centers': Stores information about the distribution centers for the StyleHub website, including their geographical locations.
Columns:
  - id (INTEGER): Unique identifier for each distribution center. (Primary Key)
  - name (STRING): Name of the distribution center.
  - latitude (FLOAT): Geographical latitude of the distribution center.
  - longitude (FLOAT): Geographical longitude of the distribution center.

